In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:,.6f}'.format

In [3]:
RETRIEVALS_FOLDER="/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/validation_queries_retrievals"
SELECTED_QUERIES_FOLDER="/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/final_validation_queries"
CLUEWEB_PARTS_FOLDER="/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed"

In [4]:
BM25_MT5_RETRIEVAL="200-queries-passagens-by-BM25-mT5_v4_with_query_id.tsv"
SPLADE_RETRIEVAL="splade_pt_result_with_clueweb_id.tsv"
COLBERTX_RETRIEVAL="colbertx_all_znormalized_validation_scores_000_199.tsv"

SAMPLED_50_QUERIES="50-dataset-doccano-queries-passagens-by-BM25-mT5_with_query_id.tsv"

In [5]:
mt5_df = pd.read_csv(os.path.join(RETRIEVALS_FOLDER, BM25_MT5_RETRIEVAL), sep="\t")

In [6]:
mt5_df.head()

,query,doc_id,passage-by-bm25-mt5,query_id
0,Qual objetivo da Enel Brasil?,clueweb22-pt0000-58-10513_2,"Sendo assim, em linha com o modelo de Criação de Valor Compartilhado (Creating Shared Value - CSV) adotado pela companhia mundialmente, temos o objetivo de associar o desenvolvimento do nosso negócio às necessidades das comunidades brasileiras e realizamos diversas iniciativas para possibilitar a geração de renda, como cursos, oficinas e outros projetos sociais. Além disso, nos comprometemos explicitamente a alcançar quatro dos 17 Objetivos de Desenvolvimento Sustentável (ODS) da ONU: amplo acesso à energia limpa e acessível, apoio à educação de qualidade, contribuição para o desenvolvimento socioeconômico e combate contra as alterações climáticas. Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique aqui e conheça o nosso Grupo",100
1,Qual objetivo da Enel Brasil?,clueweb22-pt0000-81-12555_1,"A Enel Brasil foi constituída em 2005 com o objeto de centralizar as participações societárias do grupo Enel no país e criar, por meio de uma holding de empresas, uma plataforma sólida de gestão dos ativos que preparasse a companhia para futuras expansões. Em 2007, a Enel SpA, controladora indireta da Enel Brasil, iniciou a consolidação de sua participação sobre a holding brasileira. Por meio de uma oferta pública no exterior, a Enel SpA adquiriu a totalidade das ações de emissão da espanhola Endesa – na época acionista controladora indireta da Enel Brasil. Em 2009, o controle da Enel SpA sobre a Enel Brasil foi ampliado, com a aquisição de 92,06% do capital social e votante da Endesa. Finalmente, em 2014, a Enel SpA adquiriu – por meio de uma operação de compra e venda de ações – a participação da Endesa em sociedades na América Latina, o que incluiu a Enel Brasil. Desde então, a empresa espanhola foi excluída da estrutura acionária no país. Apesar da holding Enel Brasil ter nascido em 2005, a trajetória do grupo Enel no país começou quase uma década antes, em 1996, com a aquisição da distribuidora de energia elétrica CERJ em leilão de privatização. Novas aquisições foram realizadas ao longo dos anos, garantindo a ampliação e diversificação do portfólio.",100
2,Qual objetivo da Enel Brasil?,clueweb22-pt0000-39-18411_1,"Inovação e Sustentabilidade inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique aqui e conheça o nosso Grupo",100
3,Qual objetivo da Enel Brasil?,clueweb22-pt0002-00-04601_1,"inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique 

In [7]:
splade_df = pd.read_csv(os.path.join(RETRIEVALS_FOLDER, SPLADE_RETRIEVAL), sep="\t")

In [8]:
splade_df.head()

,query_id,Q0,colbertx_id,position,score,comment,clueweb_id
0,0,Q0,6124160,1,18.215471,SPLADE-PT,clueweb22-pt0001-27-16321_1
1,0,Q0,9394964,2,18.258245,SPLADE-PT,clueweb22-pt0001-97-01569_0
2,1,Q0,9394964,706,18.854155,SPLADE-PT,clueweb22-pt0001-97-01569_0
3,68,Q0,9394964,921,18.207718,SPLADE-PT,clueweb22-pt0001-97-01569_0
4,0,Q0,319340,3,17.736769,SPLADE-PT,clueweb22-pt0000-06-04821_0


In [9]:
colbertx_df = pd.read_csv(os.path.join(RETRIEVALS_FOLDER, COLBERTX_RETRIEVAL), sep='\t')

In [10]:
colbertx_df.head()

,query_id,colbertx_id,doc_position,doc_score,score_znorm,part,clueweb_id
0,0,4138910,1,28.797396,3.723518,2,clueweb22-pt0000-86-19982_33
1,1,4138910,1,28.135212,3.365722,2,clueweb22-pt0000-86-19982_33
2,25,4138910,671,21.382965,-0.282701,2,clueweb22-pt0000-86-19982_33
3,26,4138910,476,20.872330,-0.558611,2,clueweb22-pt0000-86-19982_33
4,54,4138910,31,26.231224,2.336945,2,clueweb22-pt0000-86-19982_33


In [11]:
start_pos = 0
how_many = 10

In [42]:
results = []

i = 0

for group_name, group_df in mt5_df.groupby('query_id', sort=False):
    
    print("Handling query {}...".format(group_name))
    
    query_docs = {}
    
    mt5_docs = group_df.iloc[start_pos:(start_pos + how_many)]['doc_id'].to_numpy()
    
    for doc in mt5_docs:
        query_docs[doc] = {"query_id": group_name,
                           "query": group_df['query'].iloc[0],
                           "passage_id": doc,
                           "mT5": True,
                           "splade": False,
                           "colbertx": False
                          }
    
    splade_docs = splade_df[splade_df['query_id'] == group_name].sort_values(['score'], ascending=[False]).iloc[start_pos:(start_pos + how_many)]['clueweb_id'].to_numpy()
        
    for doc in splade_docs:
        if doc in query_docs:
            
            print("doc {} already processed...")
            
            query_docs[doc]['splade'] = True
        else:
            query_docs[doc] = {"query_id": group_name,
                               "query": group_df['query'].iloc[0],
                               "passage_id": doc,
                               "mT5": False,
                               "splade": True,
                               "colbertx": False
                              }
        

    colbertx_docs = colbertx_df[colbertx_df['query_id'] == group_name].sort_values(['score_znorm'], ascending=[False]).iloc[start_pos:(start_pos + how_many)]['clueweb_id'].to_numpy()
        
    for doc in colbertx_docs:
        if doc in query_docs:
            
            print("doc {} already processed...")
            
            query_docs[doc]['colbertx'] = True
        else:
            query_docs[doc] = {"query_id": group_name,
                               "query": group_df['query'].iloc[0],
                               "passage_id": doc,
                               "mT5": False,
                               "splade": False,
                               "colbertx": True
                              }
        
        
    results += list(query_docs.values())

Handling query 100...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 101...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 102...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 103...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 104...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already process

doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 156...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 157...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 158...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} alr

doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 10...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 11...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 12...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 13...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already proce

doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 68...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 69...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 70...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
Handling query 71...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already processed...
doc {} already proce

In [43]:
len(results)

4393

In [44]:
results_df = pd.DataFrame(results).sort_values(['query_id', 'passage_id'])

In [45]:
results_df

,query_id,query,passage_id,mT5,splade,colbertx
4346,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-09-13182_0,True,True,False
4345,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-14-10117_0,True,False,False
4340,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-34-14067_2,True,False,False
4358,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-48-18850_1,False,False,True
4342,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-54-01849_0,True,False,False
...,...,...,...,...,...,...
2146,199,Quais foram os principais artistas do Impressionismo?,clueweb22-pt0001-70-17886_7,False,False,True
2144,199,Quais foram os principais artistas do Impressionismo?,clueweb22-pt0001-73-07268_17,False,True,False
2133,199,Quais foram os principais artistas do Impressionismo?,clueweb22-pt0001-80-13719_2,True,False,True
2128,199,Quais foram os principais artistas do Impressionismo?,clueweb22-pt0001-84-17796_0,True,True,True


In [46]:
results_df.groupby('query_id')['query_id'].count().describe()

count   200.000000
mean     21.965000
std       3.772923
min      14.000000
25%      19.000000
50%      22.000000
75%      25.000000
max      30.000000
Name: query_id, dtype: float64

In [47]:
id_conversion_df = pd.read_csv(os.path.join(CLUEWEB_PARTS_FOLDER, "clueweb22-pt_colbertx_to_clueweb_id.tsv"), sep="\t", names=['colbertx_id', 'clueweb_id'])

In [48]:
id_conversion_df.head()

,colbertx_id,clueweb_id
0,0,clueweb22-pt0000-00-00001_0
1,1,clueweb22-pt0000-00-00001_1
2,2,clueweb22-pt0000-00-00001_4
3,3,clueweb22-pt0000-00-00001_5
4,4,clueweb22-pt0000-00-00003_0


In [49]:
results_df = results_df.merge(id_conversion_df, left_on='passage_id', right_on='clueweb_id')[list(results_df.columns) + ['colbertx_id']]

In [50]:
results_df.head()

,query_id,query,passage_id,mT5,splade,colbertx,colbertx_id
0,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-09-13182_0,True,True,False,473093
1,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-14-10117_0,True,False,False,715475
2,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-34-14067_2,True,False,False,1665284
3,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-48-18850_1,False,False,True,2325385
4,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-54-01849_0,True,False,False,2559888


In [51]:
collection_parts = glob.glob(os.path.join(CLUEWEB_PARTS_FOLDER, "clueweb22-pt_colbertx_0*.tsv"))

In [52]:
collection_parts

['/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_01.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_00.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_02.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_03.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_04.tsv']

In [53]:
all_parts = []

for i, collection_part in enumerate(collection_parts):
    print("Merging {}...".format(os.path.basename(collection_part)))

    part_df = pd.read_csv(collection_part, sep="\t", header=None, names=['colbertx_id', 'passage'])

    all_parts.append(results_df.merge(part_df))

Merging clueweb22-pt_colbertx_01.tsv...
Merging clueweb22-pt_colbertx_00.tsv...
Merging clueweb22-pt_colbertx_02.tsv...
Merging clueweb22-pt_colbertx_03.tsv...
Merging clueweb22-pt_colbertx_04.tsv...


In [54]:
all_parts_df = pd.concat(all_parts)

In [55]:
all_parts_df.head()

,query_id,query,passage_id,mT5,splade,colbertx,colbertx_id,passage
0,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-48-18850_1,False,False,True,2325385,"O Brasil também comporta um mostruário bastante diversificado das principais paisagens ecológicas do planeta. Os biomas encontrados no país são: Caatinga, Campos, Cerrado, Floresta Amazônica, Manguezal, Mata Atlântica, Mata de Araucária, Mata de Cocais, Pantanal e Zonas Litorâneas. Confira nossa subseção e conheça as características de cada bioma brasileiro, a posição geográfica do Brasil, os climas predominantes no país, o relevo, hidrografia, entre outros elementos da Geografia Física do Brasil. Boa leitura! Por: Wagner de Cerqueria e Francisco Artigos de Geografia Física do Brasil A posição geográfica do Brasil As características geográficas do território brasileiro. Amazônia Entenda mais as características da Amazônia. Conheça sobre a fauna e a flora da região. Saiba a importância da Floresta Amazônica para o Brasil e para o mundo. Amazônia e a água no Brasil Entenda a relação de interdependência entre a Amazônia e a água no Brasil no que diz respeito à disponibilidade dos recursos hídricos no país. Aquífero Guarani Descubra como funciona um aquífero e conheça um pouco mais sobre o Aquífero Guarani, um dos maiores do mundo. As vegetações Brasileiras"
1,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-54-01849_0,True,False,False,2559888,"(UFPR 2011) O território brasileiro possui diverso... | Questões ENEM | Me Salva! Questão 8 (UFPR 2011) O território brasileiro possui diversos biomas, entre os quais destacam-se a Floresta Amazônica, o Cerrado e a Mata Atlântica. Sobre esses biomas, é correto afirmar: O cerrado, que se localiza na região central do Brasil, tem como característica formar-se em solos pobres e arenosos e, em consequência, é pouco ameaçado pela expansão agrícola. A Floresta Amazônica, formação localizada notadamente no norte do Brasil, tende a desaparecer nas próximas décadas, haja vista que o desmatamento e as queimadas têm seus índices elevados ano a ano, evidenciando a ausência de políticas públicas voltadas à conservação daquela floresta. Uma característica comum entre esses três biomas é que todos apresentam elevada biodiversidade e presença de espécies endêmicas, evidenciando que todos precisam ser igualmente preservados. A Mata Atlântica, formação que se estendia desde o litoral nordestino ao Rio Grande Sul, onde se localiza boa parte dos maiores centros brasileiros, foi o bioma mais desmatado do país, motivo pelo qual seus remanescentes foram transformados em unidades de conservação, o que lhe garante a maior extensão em áreas preservadas do Brasil."
2,54,Quais são os biomas do Brasil?,clueweb22-pt0000-54-01849_0,True,False,False,2559888,"(UFPR 2011) O território brasileiro possui diverso... | Questões ENEM | Me Salva! Questão 8 (UFPR 2011) O território brasileiro possui diversos biomas, entre os quais destacam-se a Floresta Amazônica, o Cerrado e a Mata Atlântica. Sobre esses biomas, é correto afirmar: O cerrado, que se localiza na região central do Brasil, tem como característica formar-se em solos pobres e arenosos e, em consequência, é pouco ameaçado pela expansão agrícola. A Floresta Amazônica, formação localizada notadamente no norte do Brasil, tende a desaparecer nas próximas décadas, haja vista que o desmatamento e as queimadas têm seus índices elevados ano a ano, evidenciando a ausência de políticas públicas voltadas à conservação daquela floresta. Uma característica comum entre esses três biomas é que todos apresentam elevada biodiversidade e presença de espécies endêmicas, evidenciando que todos precisam ser igualmente preservados. A Mata Atlântica, formação que se estendia desde o litoral nordestino ao Rio Grande Sul, onde se localiza boa parte dos maiores centros brasileiros, foi o bioma mais desmatado do país, motivo pelo qual seus remanescentes foram transformados em unidades de conservação, o que 

In [56]:
all_parts_df.shape

(4393, 8)

In [57]:
results_df.shape

(4393, 7)

In [58]:
all_parts_df.to_csv(os.path.join("tests", "mt5_splade_colbertx_docs_{:03}_{:03}_all_200_queries.tsv".format(start_pos, start_pos + how_many - 1)), sep="\t", index=False)

### Reload data

In [59]:
all_parts_df = pd.read_csv(os.path.join("tests", "mt5_splade_colbertx_docs_{:03}_{:03}_all_200_queries.tsv".format(start_pos, start_pos + how_many - 1)), sep="\t")

In [60]:
all_parts_df.groupby('query_id').count()

,query,passage_id,mT5,splade,colbertx,colbertx_id,passage
query_id,,,,,,,
0,26,26,26,26,26,26,26
1,21,21,21,21,21,21,21
2,22,22,22,22,22,22,22
3,16,16,16,16,16,16,16
4,20,20,20,20,20,20,20
5,25,25,25,25,25,25,25
6,20,20,20,20,20,20,20
7,18,18,18,18,18,18,18
8,15,15,15,15,15,15,15


In [62]:
all_parts_df.drop_duplicates('query')

,query_id,query,passage_id,mT5,splade,colbertx,colbertx_id,passage
0,0,Qual a maior característica da flora brasileira?,clueweb22-pt0000-48-18850_1,False,False,True,2325385,"O Brasil também comporta um mostruário bastante diversificado das principais paisagens ecológicas do planeta. Os biomas encontrados no país são: Caatinga, Campos, Cerrado, Floresta Amazônica, Manguezal, Mata Atlântica, Mata de Araucária, Mata de Cocais, Pantanal e Zonas Litorâneas. Confira nossa subseção e conheça as características de cada bioma brasileiro, a posição geográfica do Brasil, os climas predominantes no país, o relevo, hidrografia, entre outros elementos da Geografia Física do Brasil. Boa leitura! Por: Wagner de Cerqueria e Francisco Artigos de Geografia Física do Brasil A posição geográfica do Brasil As características geográficas do território brasileiro. Amazônia Entenda mais as características da Amazônia. Conheça sobre a fauna e a flora da região. Saiba a importância da Floresta Amazônica para o Brasil e para o mundo. Amazônia e a água no Brasil Entenda a relação de interdependência entre a Amazônia e a água no Brasil no que diz respeito à disponibilidade dos recursos hídricos no país. Aquífero Guarani Descubra como funciona um aquífero e conheça um pouco mais sobre o Aquífero Guarani, um dos maiores do mundo. As vegetações Brasileiras"
2,54,Quais são os biomas do Brasil?,clueweb22-pt0000-54-01849_0,True,False,False,2559888,"(UFPR 2011) O território brasileiro possui diverso... | Questões ENEM | Me Salva! Questão 8 (UFPR 2011) O território brasileiro possui diversos biomas, entre os quais destacam-se a Floresta Amazônica, o Cerrado e a Mata Atlântica. Sobre esses biomas, é correto afirmar: O cerrado, que se localiza na região central do Brasil, tem como característica formar-se em solos pobres e arenosos e, em consequência, é pouco ameaçado pela expansão agrícola. A Floresta Amazônica, formação localizada notadamente no norte do Brasil, tende a desaparecer nas próximas décadas, haja vista que o desmatamento e as queimadas têm seus índices elevados ano a ano, evidenciando a ausência de políticas públicas voltadas à conservação daquela floresta. Uma característica comum entre esses três biomas é que todos apresentam elevada biodiversidade e presença de espécies endêmicas, evidenciando que todos precisam ser igualmente preservados. A Mata Atlântica, formação que se estendia desde o litoral nordestino ao Rio Grande Sul, onde se localiza boa parte dos maiores centros brasileiros, foi o bioma mais desmatado do país, motivo pelo qual seus remanescentes foram transformados em unidades de conservação, o que lhe garante a maior extensão em áreas preservadas do Brasil."
6,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-82-13187_0,False,False,True,3917442,"Mamíferos do Brasil - Zoologia | Museu Nacional - UFRJ ➤ Início ➤ Exposições ➤ Zoologia (Acervo) ➤ Insetos ➤ Mamíferos do Brasil Mamíferos do Brasil Uma larga coleção de mamíferos da fauna brasileira. Entre os importantes representantes deste acervo de espécimes taxidermizados encontram-se grandes símbolos da diversidade biológica do Brasil. Bugio, Barbado, Guariba Bugio Cachorro-do-mato Caititu Capivara Caxinguelê, Serelepe Cervo-do-Pantanal Gato-do-Mato-pequeno Irara Lobo-Guará Lontra Mão-Pelada Muriqui Onça-Pintada Ouriço-Cacheiro, Cuandu Paca Preguiça-de-Bentinho Preguiça-de-três-dedos Rato-Coró Saruê, Gambá Sauá, Guigó Tamanduá-bandeira Tamanduá-í Tamanduá-mirim, meleta Tatu-Canastra Tatu-de-rabo-mole Tatu-Galinha Tatu-Peludo Veado Veado-mateiro « Voltar | Voltar ao Topo"
12,2,Por que os países Guiana e Suriname não são filiados a Conmebol?,clueweb22-pt0000-64-15131_0,True,False,False,3070765,"Confederação de Futebol da América do Norte, Central e Caribe – Wikipédia, a enciclopédia livre [ocultar] 71 línguas Confederação de Futebol da América do Norte, Central e Caribe Origem: Wikipédia, a enciclopédia livre. Confederação de Futebol da América do Norte, Central e Caribe (CO

In [63]:
selected_queries_df = pd.read_csv(os.path.join(SELECTED_QUERIES_FOLDER, SAMPLED_50_QUERIES), sep='\t')

In [64]:
selected_queries_df['query_id'].drop_duplicates()

0      105
10     113
20     115
30     117
40     126
50     127
60     128
70     136
80     147
90     152
100    153
110    154
120    160
130    161
140    163
150    167
160    170
170    172
180    180
190    181
200    182
210    189
220    193
230    195
240    196
250    199
260      1
270      2
280      9
290     11
300     13
310     15
320     17
330     20
340     21
350     22
360     24
370     26
380     28
390     47
400     49
410     51
420     54
430     60
440     62
450     64
460     68
470     84
480     95
490     98
Name: query_id, dtype: int64

In [65]:
only_selected_df = all_parts_df.merge(selected_queries_df['query_id'].drop_duplicates())

In [66]:
only_selected_df.groupby('query_id').count()

,query,passage_id,mT5,splade,colbertx,colbertx_id,passage
query_id,,,,,,,
1,21,21,21,21,21,21,21
2,22,22,22,22,22,22,22
9,24,24,24,24,24,24,24
11,21,21,21,21,21,21,21
13,20,20,20,20,20,20,20
15,16,16,16,16,16,16,16
17,18,18,18,18,18,18,18
20,29,29,29,29,29,29,29
21,26,26,26,26,26,26,26


In [67]:
only_selected_df['query_id'].unique().shape

(50,)

In [68]:
mt5_df

,query,doc_id,passage-by-bm25-mt5,query_id
0,Qual objetivo da Enel Brasil?,clueweb22-pt0000-58-10513_2,"Sendo assim, em linha com o modelo de Criação de Valor Compartilhado (Creating Shared Value - CSV) adotado pela companhia mundialmente, temos o objetivo de associar o desenvolvimento do nosso negócio às necessidades das comunidades brasileiras e realizamos diversas iniciativas para possibilitar a geração de renda, como cursos, oficinas e outros projetos sociais. Além disso, nos comprometemos explicitamente a alcançar quatro dos 17 Objetivos de Desenvolvimento Sustentável (ODS) da ONU: amplo acesso à energia limpa e acessível, apoio à educação de qualidade, contribuição para o desenvolvimento socioeconômico e combate contra as alterações climáticas. Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique aqui e conheça o nosso Grupo",100
1,Qual objetivo da Enel Brasil?,clueweb22-pt0000-81-12555_1,"A Enel Brasil foi constituída em 2005 com o objeto de centralizar as participações societárias do grupo Enel no país e criar, por meio de uma holding de empresas, uma plataforma sólida de gestão dos ativos que preparasse a companhia para futuras expansões. Em 2007, a Enel SpA, controladora indireta da Enel Brasil, iniciou a consolidação de sua participação sobre a holding brasileira. Por meio de uma oferta pública no exterior, a Enel SpA adquiriu a totalidade das ações de emissão da espanhola Endesa – na época acionista controladora indireta da Enel Brasil. Em 2009, o controle da Enel SpA sobre a Enel Brasil foi ampliado, com a aquisição de 92,06% do capital social e votante da Endesa. Finalmente, em 2014, a Enel SpA adquiriu – por meio de uma operação de compra e venda de ações – a participação da Endesa em sociedades na América Latina, o que incluiu a Enel Brasil. Desde então, a empresa espanhola foi excluída da estrutura acionária no país. Apesar da holding Enel Brasil ter nascido em 2005, a trajetória do grupo Enel no país começou quase uma década antes, em 1996, com a aquisição da distribuidora de energia elétrica CERJ em leilão de privatização. Novas aquisições foram realizadas ao longo dos anos, garantindo a ampliação e diversificação do portfólio.",100
2,Qual objetivo da Enel Brasil?,clueweb22-pt0000-39-18411_1,"Inovação e Sustentabilidade inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique aqui e conheça o nosso Grupo",100
3,Qual objetivo da Enel Brasil?,clueweb22-pt0002-00-04601_1,"inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique 

In [69]:
only_selected_df

,query_id,query,passage_id,mT5,splade,colbertx,colbertx_id,passage
0,54,Quais são os biomas do Brasil?,clueweb22-pt0000-54-01849_0,True,False,False,2559888,"(UFPR 2011) O território brasileiro possui diverso... | Questões ENEM | Me Salva! Questão 8 (UFPR 2011) O território brasileiro possui diversos biomas, entre os quais destacam-se a Floresta Amazônica, o Cerrado e a Mata Atlântica. Sobre esses biomas, é correto afirmar: O cerrado, que se localiza na região central do Brasil, tem como característica formar-se em solos pobres e arenosos e, em consequência, é pouco ameaçado pela expansão agrícola. A Floresta Amazônica, formação localizada notadamente no norte do Brasil, tende a desaparecer nas próximas décadas, haja vista que o desmatamento e as queimadas têm seus índices elevados ano a ano, evidenciando a ausência de políticas públicas voltadas à conservação daquela floresta. Uma característica comum entre esses três biomas é que todos apresentam elevada biodiversidade e presença de espécies endêmicas, evidenciando que todos precisam ser igualmente preservados. A Mata Atlântica, formação que se estendia desde o litoral nordestino ao Rio Grande Sul, onde se localiza boa parte dos maiores centros brasileiros, foi o bioma mais desmatado do país, motivo pelo qual seus remanescentes foram transformados em unidades de conservação, o que lhe garante a maior extensão em áreas preservadas do Brasil."
1,54,Quais são os biomas do Brasil?,clueweb22-pt0000-77-06884_1,False,True,False,3668516,"O tamanho do Brasil e a grande variedade de climas e características e suas regiões são o que torna possível existir no país um número tão alto de espécies. Dentre os biomas brasileiros, os maiores em biodiversidade são a Amazônia, a Mata Atlântica e o Cerrado. Este post é inspirado no livro infantil “Jornada pela Floresta” No livro personalizado Uma Jornada pela Floresta, as crianças são apresentadas de forma divertida e lúdica a vários animais típicos do Brasil: boto-cor-de-rosa, mico-leão-dourado, arara-azul, sucuri, ariranha e outros mais. Logo a seguir, você pode conferir curiosidades sobre cada espécie para compartilhar com os pequenos e aprender mais sobre esses animais que todos nós temos o dever de cuidar e proteger! Boto-cor-de-rosa O boto-cor-de-rosa nasce cinza e muda de cor com o passar do tempo. O boto-cor-de-rosa é um mamífero que vive na água. Ele é parente das baleias e dos golfinhos: a diferença é que ele não vive no mar, mas sim na água doce. Seu habitat são os rios amazônicos do Brasil, da Bolívia, da Venezuela, da Colômbia, do Equador e do Peru."
2,54,Quais são os biomas do Brasil?,clueweb22-pt0000-54-07613_0,True,False,False,2574108,"Biomas Brasileiros: tipos e resumo - Toda Matéria Biologia › Ecologia Biomas Brasileiros Lana Magalhães Professora de Biologia São seis os grandes biomas brasileiros (continentais). Os biomas são conjuntos de ecossistemas (vegetal e animal) com uma diversidade biológica própria. Segundo o IBGE, no Brasil há seis tipos de biomas continentais e um bioma marinho ou aquático. Quais são os biomas terrestres brasileiros, então? Amazônia Cerrado Caatinga Mata Atlântica Pantanal Pampa Mapa de Biomas do Brasil Observe no mapa onde estão localizados os biomas terrestres brasileiros. Biomas Terrestres do Brasil Por suas dimensões continentais, o país abriga biomas tão distintos entre si como florestas tropicais até a vegetação cerrada. Conheça um pouco mais dessa diversidade: Bioma Amazônia Bioma Amazônia: rios, mata fechada e uma diversidade biológica insuperável Considerado o maior Bioma brasileiro e a maior reserva de diversidade biológica do mundo, o bioma Amazônia corresponde a quase metade do território nacional. Abrange os estados brasileiros do: Acre, Amapá, Amazonas, Pará, Roraima; parte de Rondônia, Mato Grosso, Maranhão e Tocantins. O clima dessa região é quente e úmido e sua densa vegetação é caracterizada pela floresta amazônica com árvores de grande porte. Leia também:"
3,54,Quais são os biomas d

In [70]:
mt5_df[mt5_df['doc_id'] == 'clueweb22-pt0000-54-01849_0']

,query,doc_id,passage-by-bm25-mt5,query_id
145003,Quais são os biomas do Brasil?,clueweb22-pt0000-54-01849_0,"(UFPR 2011) O território brasileiro possui diverso... | Questões ENEM | Me Salva! Questão 8 (UFPR 2011) O território brasileiro possui diversos biomas, entre os quais destacam-se a Floresta Amazônica, o Cerrado e a Mata Atlântica. Sobre esses biomas, é correto afirmar: O cerrado, que se localiza na região central do Brasil, tem como característica formar-se em solos pobres e arenosos e, em consequência, é pouco ameaçado pela expansão agrícola. A Floresta Amazônica, formação localizada notadamente no norte do Brasil, tende a desaparecer nas próximas décadas, haja vista que o desmatamento e as queimadas têm seus índices elevados ano a ano, evidenciando a ausência de políticas públicas voltadas à conservação daquela floresta. Uma característica comum entre esses três biomas é que todos apresentam elevada biodiversidade e presença de espécies endêmicas, evidenciando que todos precisam ser igualmente preservados. A Mata Atlântica, formação que se estendia desde o litoral nordestino ao Rio Grande Sul, onde se localiza boa parte dos maiores centros brasileiros, foi o bioma mais desmatado do país, motivo pelo qual seus remanescentes foram transformados em unidades de conservação, o que lhe garante a maior extensão em áreas preservadas do Brasil.",54
198004,Qual a maior característica da flora brasileira?,clueweb22-pt0000-54-01849_0,"(UFPR 2011) O território brasileiro possui diverso... | Questões ENEM | Me Salva! Questão 8 (UFPR 2011) O território brasileiro possui diversos biomas, entre os quais destacam-se a Floresta Amazônica, o Cerrado e a Mata Atlântica. Sobre esses biomas, é correto afirmar: O cerrado, que se localiza na região central do Brasil, tem como característica formar-se em solos pobres e arenosos e, em consequência, é pouco ameaçado pela expansão agrícola. A Floresta Amazônica, formação localizada notadamente no norte do Brasil, tende a desaparecer nas próximas décadas, haja vista que o desmatamento e as queimadas têm seus índices elevados ano a ano, evidenciando a ausência de políticas públicas voltadas à conservação daquela floresta. Uma característica comum entre esses três biomas é que todos apresentam elevada biodiversidade e presença de espécies endêmicas, evidenciando que todos precisam ser igualmente preservados. A Mata Atlântica, formação que se estendia desde o litoral nordestino ao Rio Grande Sul, onde se localiza boa parte dos maiores centros brasileiros, foi o bioma mais desmatado do país, motivo pelo qual seus remanescentes foram transformados em unidades de conservação, o que lhe garante a maior extensão em áreas preservadas do Brasil.",0


In [71]:
mt5_df[mt5_df['doc_id'] == 'clueweb22-pt0001-52-19251_1']

,query,doc_id,passage-by-bm25-mt5,query_id
72008,Como configurar uma linguagem do teclado windowns 10?,clueweb22-pt0001-52-19251_1,"Aparecerá a opção de todos os idiomas, selecione e clique à correspondente ao seu teclado. Após isso, dependendo do idioma, terá a opção de selecionar o local do idioma, como no caso do Português. Pronto! O seu está com o teclado configurado! Configurando a linguagem no Windows 8 Acesse o Painel de controle e clique em Idioma. Selecione Adicionar idioma. Aparecerá uma lista com os idiomas, clique no correspondente do seu teclado. Dependendo do idioma, vai aparecer uma janela para escolher o local do idioma, assim como no Windows 10.Selecione o local correto e clique em Adicionar. Se houver outro idioma configurado, mova para cima, qual que você configure o qual deseja ser o padrão. Para saber como configurar a linguagem e layout no Windows 7 acesse aqui. 😉 Layout Aqui no Brasil, temos o ABNT e o ABNT2, que é a maneira que o teclado está mapeado. Se você quer descobrir qual é o layout do seu teclado , clique aqui. No ABNT2, existe a tecla “ Alt Gr ” que fica ao lado direito do espaço e, permite ao usuário utilizar a terceira função das teclas, como por exemplo: ²,³,£,¢,¬. Dica bringIT: Se você apertar ao mesmo tempo as teclas Windows e Espaço, aparecerá as opções de layout configuradas da sua máquina. Configurando o layout no Windows 10",172


In [72]:
only_selected_df.to_csv(os.path.join("tests", "mt5_splade_colbertx_docs_{:03}_{:03}_sampled_50_queries.tsv".format(start_pos, start_pos + how_many - 1)), sep="\t", index=False)

### Save single query

In [76]:
only_selected_df[only_selected_df['query_id'] == 105].to_csv(os.path.join("tests", "mt5_splade_colbertx_docs_query_105.tsv"), sep="\t", index=False)

### Check the selected questions/passages

In [ ]:
only_selected_df = pd.read_csv(os.path.join("tests", "mt5_splade_colbertx_docs_{:03}_{:03}_sampled_50_queries.tsv".format(start_pos, start_pos + how_many - 1)), sep="\t")

In [ ]:
only_selected_df.groupby('query_id')['passage_id'].count().describe()

In [ ]:
p1_df = pd.read_csv("tests/mt5_splade_colbertx_docs_000_001_sampled_50_queries.tsv", sep='\t')

In [ ]:
p2_df = pd.read_csv("tests/mt5_splade_colbertx_docs_002_009_sampled_50_queries.tsv", sep='\t')

In [ ]:
p1_df.shape

In [ ]:
p2_df.shape

In [ ]:
only_selected_df.shape

In [ ]:
p1_df.shape[0] + p2_df.shape[0]

In [ ]:
p1_df[(p1_df['query_id'] == 1) & (p1_df['mT5'])]['passage_id'].to_numpy()

In [ ]:
p2_df[(p2_df['query_id'] == 1) & (p2_df['mT5'])]['passage_id'].to_numpy()

In [ ]:
passages = np.concatenate([p1_df[(p1_df['query_id'] == 1) & (p1_df['mT5'])]['passage_id'].to_numpy(), p2_df[(p2_df['query_id'] == 1) & (p2_df['mT5'])]['passage_id'].to_numpy()])

In [ ]:
passages.shape

In [ ]:
np.intersect1d(passages, only_selected_df[(only_selected_df['query_id'] == 1) & (only_selected_df['mT5'])]['passage_id'].to_numpy()).shape

In [ ]:
only_selected_df[(only_selected_df['query_id'] == 1) & (only_selected_df['mT5'])]['passage_id'].to_numpy().shape

In [ ]:
only_selected_df

### Check evaluation results

In [77]:
test_df = pd.read_csv("tests/evaluation_query_105_gpt-4-1106-preview_20231129.tsv", sep='\t')

In [79]:
test_df.shape

(21, 14)

In [105]:
ref_df = pd.read_csv("tests/test_000_119_gpt-4-1106-preview_20231108_fixed_2_scores.tsv", sep='\t')

In [106]:
ref_df

,doccano_id,query,passage,passage_id,score,reasoning,usage,cost,duration,saved_cost
0,4101,Onde está localizada a Praça XV de Novembro?,"Praça XV (Rio de Janeiro) – Wikipédia, a enciclopédia livre [ocultar] 5 línguas Praça XV (Rio de Janeiro) Origem: Wikipédia, a enciclopédia livre. Coordenadas: 22° 54′ 10″ S, 43° 10′ 24″ W Nota: Este artigo é sobre a praça da cidade do Rio de Janeiro. Para praças de outros lugares, veja Praça XV de Novembro. Esta página cita fontes, mas estas não cobrem todo o conteúdo. Ajude a inserir referências. Conteúdo não verificável poderá ser removido .— Encontre fontes: Google (notícias, livros e acadêmico)(Maio de 2019) A praça e as edificações das imediações. Trecho do Elevado da Perimetral que passava sobre a praça, em 2011. A Praça XV, também conhecida como Praça XV de Novembro, Praça Quinze de Novembro ou Praça Quinze, é uma praça situada no bairro do Centro, na Zona Central da cidade do Rio de Janeiro. Além de estar localizada na Rua Primeiro de Março, a praça integra a Orla Conde, um passeio público que margeia a Baía de Guanabara . A praça, que existe desde o século XVI, foi reinaugurada no dia 29 de maio de 2016 após ser revitalizada.[1] A revitalização da praça foi feita durante a gestão do prefeito Eduardo Paes no âmbito do Porto Maravilha, uma operação urbana que visa revitalizar a Zona Portuária do Rio de Janeiro.",clueweb22-pt0001-14-16263_0,3,"A passagem responde diretamente à pergunta, fornecendo a localização exata da Praça XV de Novembro, que é no bairro do Centro, na Zona Central da cidade do Rio de Janeiro, e menciona que está localizada na Rua Primeiro de Março e integra a Orla Conde.","{'prompt_tokens': 887, 'completion_tokens': 81, 'total_tokens': 968}",0.031470,2.224712,0.000000
1,4102,Onde está localizada a Praça XV de Novembro?,"Rua XV de Novembro, 818 Centro, Pomerode, Santa Catarina 89107-000 Brasil Tela cheia O melhor por perto Restaurantes 100 dentro de 5 km Tarthurel Pizzaria 317 56 m $$ - $$$ • Italiana • Pizza Restaurante Torten Paradies 1.409 0.5 km $$ - $$$ • Docerias • Alemã • Brasileira Mon Cheri Bistro @ Casa Romena 106 0.7 km $$ - $$$ • Mediterrânea • Saudável • Leste europeia Veja tudo Atrações 24 dentro de 10 km Praça Jorge Lacerda 58 0.1 km Pontos de interesse Parque Municipal de Eventos 25 0.3 km Centros de convenção Parque Educativo Vila Encantada 1.057 0.9 km Parques Veja tudo Contribua Faça uma avaliação Carregue uma foto Avaliações Mais recente 4.5 722 avaliações Excelente 379 Muito boa 266 Razoável 68 Ruim 9 Horrível 1 Thais Leme, SP 161 contribuições Parada obrigatória para fotos mar de 2022 Quando eu via as fotos achava que era bem maior. O portal é muito bonito e tem um centro e informações turísticas. Estava no período da Osterfest, então estava todo com a decoração de Páscoa. Feita em 29 de abril de 2022 Esta avaliação representa a opinião subjetiva de um membro do Tripadvisor, e não da TripAdvisor LLC. daninoguei São Paulo, SP 2.143 contribuições MUITAS PESSOAS PARA TIRAR FOTOS mar de 2022 Portal turístico da cidade, local de parada para fotos com a representação da arquitetura local.",clueweb22-pt0000-27-16948_2,3,"A passagem fornece o endereço completo da Praça XV de Novembro, que está localizada em Pomerode, Santa Catarina, Brasil, respondendo diretamente à pergunta feita.","{'prompt_tokens': 903, 'completion_tokens': 54, 'total_tokens': 957}",0.030330,1.806348,0.000000
2,4103,Onde está localizada a Praça XV de Novembro?,"A praça recebeu esse nome por homenagear o dia 15 de novembro de 1889, marcado pela Proclamação da República do Brasil. Nesse dia, foi instaurada a república federativa presidencialista no Brasil, em substituição à monarquia constitucional parlamentarista em vigor desde 1822. Índice 1 História 1.1 Período colonial 1.2 Século XIX 1.3 Século XX 1.4 Século XXI 2 Esculturas e Monumentos 2.1 Chafariz do Mestre Valentim 2.2 Estátua Equestre de Dom João VI 2.3 Estátua Equestre do General Osório 2.4 Estátua do Almirante Negro 2.5 Pisk

In [107]:
ref_scores_df = ref_df[ref_df['query'] == test_df.iloc[0]['query']]

In [108]:
ref_scores_df

,doccano_id,query,passage,passage_id,score,reasoning,usage,cost,duration,saved_cost
0,4101,Onde está localizada a Praça XV de Novembro?,"Praça XV (Rio de Janeiro) – Wikipédia, a enciclopédia livre [ocultar] 5 línguas Praça XV (Rio de Janeiro) Origem: Wikipédia, a enciclopédia livre. Coordenadas: 22° 54′ 10″ S, 43° 10′ 24″ W Nota: Este artigo é sobre a praça da cidade do Rio de Janeiro. Para praças de outros lugares, veja Praça XV de Novembro. Esta página cita fontes, mas estas não cobrem todo o conteúdo. Ajude a inserir referências. Conteúdo não verificável poderá ser removido .— Encontre fontes: Google (notícias, livros e acadêmico)(Maio de 2019) A praça e as edificações das imediações. Trecho do Elevado da Perimetral que passava sobre a praça, em 2011. A Praça XV, também conhecida como Praça XV de Novembro, Praça Quinze de Novembro ou Praça Quinze, é uma praça situada no bairro do Centro, na Zona Central da cidade do Rio de Janeiro. Além de estar localizada na Rua Primeiro de Março, a praça integra a Orla Conde, um passeio público que margeia a Baía de Guanabara . A praça, que existe desde o século XVI, foi reinaugurada no dia 29 de maio de 2016 após ser revitalizada.[1] A revitalização da praça foi feita durante a gestão do prefeito Eduardo Paes no âmbito do Porto Maravilha, uma operação urbana que visa revitalizar a Zona Portuária do Rio de Janeiro.",clueweb22-pt0001-14-16263_0,3,"A passagem responde diretamente à pergunta, fornecendo a localização exata da Praça XV de Novembro, que é no bairro do Centro, na Zona Central da cidade do Rio de Janeiro, e menciona que está localizada na Rua Primeiro de Março e integra a Orla Conde.","{'prompt_tokens': 887, 'completion_tokens': 81, 'total_tokens': 968}",0.031470,2.224712,0.000000
1,4102,Onde está localizada a Praça XV de Novembro?,"Rua XV de Novembro, 818 Centro, Pomerode, Santa Catarina 89107-000 Brasil Tela cheia O melhor por perto Restaurantes 100 dentro de 5 km Tarthurel Pizzaria 317 56 m $$ - $$$ • Italiana • Pizza Restaurante Torten Paradies 1.409 0.5 km $$ - $$$ • Docerias • Alemã • Brasileira Mon Cheri Bistro @ Casa Romena 106 0.7 km $$ - $$$ • Mediterrânea • Saudável • Leste europeia Veja tudo Atrações 24 dentro de 10 km Praça Jorge Lacerda 58 0.1 km Pontos de interesse Parque Municipal de Eventos 25 0.3 km Centros de convenção Parque Educativo Vila Encantada 1.057 0.9 km Parques Veja tudo Contribua Faça uma avaliação Carregue uma foto Avaliações Mais recente 4.5 722 avaliações Excelente 379 Muito boa 266 Razoável 68 Ruim 9 Horrível 1 Thais Leme, SP 161 contribuições Parada obrigatória para fotos mar de 2022 Quando eu via as fotos achava que era bem maior. O portal é muito bonito e tem um centro e informações turísticas. Estava no período da Osterfest, então estava todo com a decoração de Páscoa. Feita em 29 de abril de 2022 Esta avaliação representa a opinião subjetiva de um membro do Tripadvisor, e não da TripAdvisor LLC. daninoguei São Paulo, SP 2.143 contribuições MUITAS PESSOAS PARA TIRAR FOTOS mar de 2022 Portal turístico da cidade, local de parada para fotos com a representação da arquitetura local.",clueweb22-pt0000-27-16948_2,3,"A passagem fornece o endereço completo da Praça XV de Novembro, que está localizada em Pomerode, Santa Catarina, Brasil, respondendo diretamente à pergunta feita.","{'prompt_tokens': 903, 'completion_tokens': 54, 'total_tokens': 957}",0.030330,1.806348,0.000000
2,4103,Onde está localizada a Praça XV de Novembro?,"A praça recebeu esse nome por homenagear o dia 15 de novembro de 1889, marcado pela Proclamação da República do Brasil. Nesse dia, foi instaurada a república federativa presidencialista no Brasil, em substituição à monarquia constitucional parlamentarista em vigor desde 1822. Índice 1 História 1.1 Período colonial 1.2 Século XIX 1.3 Século XX 1.4 Século XXI 2 Esculturas e Monumentos 2.1 Chafariz do Mestre Valentim 2.2 Estátua Equestre de Dom João VI 2.3 Estátua Equestre do General Osório 2.4 Estátua do Almirante Negro 2.5 Pisk

In [109]:
test_df.merge(ref_scores_df, left_on="passage_id", right_on="passage_id")

,query_id,query_x,passage_id,mT5,splade,colbertx,colbertx_id,passage_x,score_x,reasoning_x,...,saved_cost_x,doccano_id,query_y,passage_y,score_y,reasoning_y,usage_y,cost_y,duration_y,saved_cost_y
0,105,Onde está localizada a Praça XV de Novembro?,clueweb22-pt0000-44-08794_4,True,True,True,2107121,"No local onde Floripa foi inicialmente chamada de Nossa Senhora do Desterro, foi fundada, bem no centro da cidade, a Praça XV de Novembro e sua famosa e centenária figueira. Palco de diversas manifestações políticas e culturais, atualmente é um oásis verde no meio da cidade, ótimo para relaxar e participar dos diversos eventos que acontecem ao longo do ano. Pouco acima da Praça, localiza-se a Catedral Metropolitana de Florianópolis. Construída entre 1753 e 1773 em estilo neoclássico, ela é tombada pelo Instituto do Patrimônio Histórico e Artístico Nacional, sendo aberta à visitação e cujos horários das missas e celebrações podem ser conferidos em seu site. Não deixe de visitar a Catedral Metropolitana! Foto: Carlos Colonetti Neto 2.Passeios em Florianópolis: Memorial Miramar, Alfândega e Mercado Público Saindo da Praça XV e indo na direção oposta à da Catedral, você chegará ao Memorial Miramar, que marca o local onde ficava o píer de desembarque de passageiros, no início do Século XX, antes de ser realizado o aterro da Baía Sul.",3,"A passagem responde diretamente à pergunta, indicando que a Praça XV de Novembro está localizada bem no centro da cidade de Florianópolis, sem incluir informações não relacionadas.",...,0.000000,4105,Onde está localizada a Praça XV de Novembro?,"No local onde Floripa foi inicialmente chamada de Nossa Senhora do Desterro, foi fundada, bem no centro da cidade, a Praça XV de Novembro e sua famosa e centenária figueira. Palco de diversas manifestações políticas e culturais, atualmente é um oásis verde no meio da cidade, ótimo para relaxar e participar dos diversos eventos que acontecem ao longo do ano. Pouco acima da Praça, localiza-se a Catedral Metropolitana de Florianópolis. Construída entre 1753 e 1773 em estilo neoclássico, ela é tombada pelo Instituto do Patrimônio Histórico e Artístico Nacional, sendo aberta à visitação e cujos horários das missas e celebrações podem ser conferidos em seu site. Não deixe de visitar a Catedral Metropolitana! Foto: Carlos Colonetti Neto 2.Passeios em Florianópolis: Memorial Miramar, Alfândega e Mercado Público Saindo da Praça XV e indo na direção oposta à da Catedral, você chegará ao Memorial Miramar, que marca o local onde ficava o píer de desembarque de passageiros, no início do Século XX, antes de ser realizado o aterro da Baía Sul.",3,"A passagem responde diretamente à pergunta, indicando que a Praça XV de Novembro está localizada bem no centro da cidade de Florianópolis.","{'prompt_tokens': 792, 'completion_tokens': 45, 'total_tokens': 837}",0.026460,2.305490,0.000000
1,105,Onde está localizada a Praça XV de Novembro?,clueweb22-pt0000-27-16948_2,True,False,False,1336659,"Rua XV de Novembro, 818 Centro, Pomerode, Santa Catarina 89107-000 Brasil Tela cheia O melhor por perto Restaurantes 100 dentro de 5 km Tarthurel Pizzaria 317 56 m $$ - $$$ • Italiana • Pizza Restaurante Torten Paradies 1.409 0.5 km $$ - $$$ • Docerias • Alemã • Brasileira Mon Cheri Bistro @ Casa Romena 106 0.7 km $$ - $$$ • Mediterrânea • Saudável • Leste europeia Veja tudo Atrações 24 dentro de 10 km Praça Jorge Lacerda 58 0.1 km Pontos de interesse Parque Municipal de Eventos 25 0.3 km Centros de convenção Parque Educativo Vila Encantada 1.057 0.9 km Parques Veja tudo Contribua Faça uma avaliação Carregue uma foto Avaliações Mais recente 4.5 722 avaliações Excelente 379 Muito boa 266 Razoável 68 Ruim 9 Horrível 1 Thais Leme, SP 161 contribuições Parada obrigatória para fotos mar de 2022 Quando eu via as fotos achava que era bem maior. O portal é muito bonito e tem um centro e informações turísticas. Estava no período da Osterfest, então estava todo com a decoração de Páscoa. Feita em 29 de abril 